# Per Node Permutation

In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

from tqdm.notebook import tqdm

from joblib import Parallel, delayed

data_df = None
if os.path.isfile("data/data.pkl"):
    data_df = pd.read_pickle("data/data.pkl")
else:
    pass
data_df.head()

RANDOM_STATE = 42
SPECIFIC_NODES = [2, 7, 83, 86, 120, 167]
TEST_SIZE = 0.2

In [14]:
def loocv_classification(X, y):
    loo = LeaveOneOut()
    y_true, y_pred = [], []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = SVC(kernel="sigmoid", probability=True, random_state=RANDOM_STATE)
        clf.fit(X_train, y_train)
        y_pred.append(clf.predict(X_test)[0])
        y_true.append(y_test[0])
    return accuracy_score(y_true, y_pred)

In [15]:
# Define X and Y
ad_hc_df = data_df[
    data_df["Diagnosis"].isin(["AD", "HC"]) & (data_df["Harmonized"].notna())
].copy()
tbi_df = data_df[
    data_df["Diagnosis"].isin(["NEG", "POS"]) & (data_df["Harmonized"].notna())
].copy()

X_ad_hc = np.vstack(ad_hc_df["EVC"].values)
X_tbi = np.vstack(tbi_df["EVC"].values)

X_ad_hc = StandardScaler().fit_transform(X_ad_hc)
X_tbi = StandardScaler().fit_transform(X_tbi)

y_ad_hc = ad_hc_df["Diagnosis"].map({"AD": 1, "HC": 0}).values
y_tbi = tbi_df["Diagnosis"].map({"POS": 1, "NEG": 0}).values

In [16]:
# Prepare testing holdouts
X_ad_train, X_ad_test, y_ad_train, y_ad_test = train_test_split(
    X_ad_hc, y_ad_hc, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_ad_hc
)

X_tbi_train, X_tbi_test, y_tbi_train, y_tbi_test = train_test_split(
    X_tbi, y_tbi, test_size=TEST_SIZE, stratify=y_tbi, random_state=RANDOM_STATE
)

In [17]:
# Obtain a model for AD/HC
clf = SVC(
    kernel="sigmoid", class_weight="balanced", probability=True, random_state=RANDOM_STATE
)

svc_params = {
    "C": np.logspace(-2, 2, 10),
    "kernel": ["linear", "rbf", "sigmoid"],
    "gamma": np.logspace(-4, 1, 10)
}

gridsearch = GridSearchCV(
    estimator=clf,
    param_grid=svc_params,
    cv=15,
    refit=True,
    verbose=0,
    n_jobs=-1,
    error_score='raise',
)

gridsearch.fit(X_ad_train[:, SPECIFIC_NODES], y_ad_train)
best_model_ad = gridsearch.best_estimator_

y_pred = best_model_ad.predict(X_ad_test[:, SPECIFIC_NODES])
print(classification_report(y_ad_test, y_pred))
print(gridsearch.best_params_)
score_ad = gridsearch.score(X_ad_test[:, SPECIFIC_NODES], y_ad_test)
print(score_ad)

KeyboardInterrupt: 

In [ ]:
# Obtain a model for TBI+/TBI-
print('=== OPTIMIZED MODEL ===')
gridsearch = clone(gridsearch)
gridsearch.fit(X_tbi_train[:, SPECIFIC_NODES], y_tbi_train)
best_model_tbi = gridsearch.best_estimator_

y_pred = best_model_tbi.predict(X_tbi_test[:, SPECIFIC_NODES])
print(classification_report(y_tbi_test, y_pred))
score_tbi = best_model_tbi.score(X_tbi_test[:, SPECIFIC_NODES], y_tbi_test)
print(score_tbi)

print('\n=== PARAMETER ONLY MODEL ===')
parameter_only_tbi = SVC(**best_model_ad.get_params())
parameter_only_tbi.fit(X_tbi_train[:, SPECIFIC_NODES], y_tbi_train)
y_pred = parameter_only_tbi.predict(X_tbi_test[:, SPECIFIC_NODES])
print(classification_report(y_tbi_test, y_pred))
score_param_tbi = parameter_only_tbi.score(X_tbi_test[:, SPECIFIC_NODES], y_tbi_test)
print(score_param_tbi)

=== OPTIMIZED MODEL ===
              precision    recall  f1-score   support

           0       0.43      0.50      0.46         6
           1       0.25      0.20      0.22         5

    accuracy                           0.36        11
   macro avg       0.34      0.35      0.34        11
weighted avg       0.35      0.36      0.35        11

0.36363636363636365

=== PARAMETER ONLY MODEL ===
              precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.50      0.40      0.44         5

    accuracy                           0.55        11
   macro avg       0.54      0.53      0.53        11
weighted avg       0.54      0.55      0.54        11

0.5454545454545454


In [ ]:
# Apply the AD/HC model onto the TBI+/TBI- and obtain Classification Report
y_pred = best_model_ad.predict(X_tbi[:,SPECIFIC_NODES])
print(classification_report(y_tbi, y_pred))
score_transfer = best_model_ad.score(X_tbi[:, SPECIFIC_NODES], y_tbi)
print(score_transfer)

              precision    recall  f1-score   support

           0       0.46      0.46      0.46        28
           1       0.44      0.44      0.44        27

    accuracy                           0.45        55
   macro avg       0.45      0.45      0.45        55
weighted avg       0.45      0.45      0.45        55

0.45454545454545453


In [ ]:
# Permutation of Nodes
# Each iteration, we will change the nodes used for X
from src.helper import tqdm_joblib

NUM_PERMUTATIONS = 1000
NUM_NODES = len(SPECIFIC_NODES)

def run_permutation(seed):
    rng = np.random.default_rng(seed)
    perm_nodes = rng.choice(X_ad_train.shape[1], size=NUM_NODES, replace=False)

    grid_ad = clone(gridsearch)
    grid_ad.fit(X_ad_train[:, perm_nodes], y_ad_train)
    perm_ad_score = grid_ad.score(X_ad_test[:, perm_nodes], y_ad_test)
    perm_transfer_score = grid_ad.score(X_tbi_test[:, perm_nodes], y_tbi_test)

    grid_tbi = clone(gridsearch)
    grid_tbi.fit(X_tbi_train[:, perm_nodes], y_tbi_train)
    perm_tbi_score = grid_tbi.score(X_tbi_test[:, perm_nodes], y_tbi_test)

    return {
        "Nodes": perm_nodes,
        "AD Score": perm_ad_score,
        "TBI Score": perm_tbi_score,
        "Transfer Score": perm_transfer_score
    }

with tqdm_joblib(tqdm(range(NUM_PERMUTATIONS), desc="Permutations")) as progress_bar:
    results = Parallel(n_jobs=-1)(
        delayed(run_permutation)(i) for i in range(NUM_PERMUTATIONS)
    )

perm_df = pd.DataFrame(results)
print(perm_df)

Permutations:   0%|          | 0/1000 [00:00<?, ?it/s]

### Generate a figure with histograms to visualize all of the above

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# AD Score
axes[0].hist(perm_df["AD Score"], bins=30, color='skyblue', edgecolor='black')
axes[0].axvline(score_ad, color='red', linestyle='--', label=f"True Score = {score_ad:.2f}")
axes[0].set_title("AD/HC Score Distribution")
axes[0].set_xlabel("Score")
axes[0].set_ylabel("Frequency")
axes[0].set_xlim(0,1)
axes[0].grid(True)
axes[0].legend()

# TBI Score
axes[1].hist(perm_df["TBI Score"], bins=30, color='lightgreen', edgecolor='black')
axes[1].axvline(score_tbi, color='red', linestyle='--', label=f"True Score = {score_tbi:.2f}")
axes[1].set_title("TBI+/TBI- Score Distribution")
axes[1].set_xlabel("Score")
axes[1].set_ylabel("Frequency")
axes[1].set_xlim(0,1)
axes[1].grid(True)
axes[1].legend()

# Transfer Score
axes[2].hist(perm_df["Transfer Score"], bins=30, color='plum', edgecolor='black')
axes[2].axvline(score_transfer, color='red', linestyle='--', label=f"True Score = {score_transfer:.2f}")
axes[2].set_title("Transfer Score Distribution")
axes[2].set_xlabel("Score")
axes[2].set_ylabel("Frequency")
axes[2].set_xlim(0,1)
axes[2].grid(True)
axes[2].legend()

plt.tight_layout()
plt.show()
